In [1]:
import pandas as pd
from jseg.jieba import Jieba
j = Jieba()

diaries = pd.read_csv('data/diary_corpus.csv').fillna('')
diaries = diaries[diaries.author=='楊基振日記']

DEBUG:jseg.jieba:loading default dictionary


FileNotFoundError: File b'data/diary_corpus.csv' does not exist

# Remove Japanese

In [ ]:
diaries = pd.read_csv('data/diary_corpus.csv').fillna('')
diaries = diaries[diaries.author =='楊基振日記']
days = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日', '星期天']
ls = []
out  = []
for i in diaries.index:
    content = diaries.content[i]
    title = diaries.title[i]
    for day in days:
        ind = content.find(day+'\n')
        if ind > 0:
            content = content[ind+3:]
            break
    ls.append(len(content + title))
    out.append({'title': title, 'content': content.split('。')})

# Named Entities by CJ

In [ ]:
with open('data/names.csv') as f:
    names = set(name.strip() for name in f)
with open('data/place.csv') as f:
    places = set(place.strip() for place in f)

with open('data/names.txt') as f:
    names.update(name.strip() for name in f)
with open('data/first.txt') as f:
    names.update(name.strip() for name in f)
with open('data/last.txt') as f:
    names.update(name.strip() for name in f)
with open('data/relatives.txt') as f:
    names.update(name.strip() for name in f)

# Extract Time Frames

In [ ]:
time_trigger = ['下午', '早晨', '晚上', '早上', '午夜', '今天', '明天'] + [num + '點' for num in ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十', '十一', '十二']]
frames = []
for o in out:
    for content in o['content']:
        for trigger in time_trigger:
            if trigger in content:
                frames.append({'time': o['title'], 'content': content})
                break

# Raw Format

In [ ]:
for frame in frames:
    result = j.seg(frame['content'], pos=True)
    frame['participant'] = []
    frame['place'] = []
    frame['relative_time'] = []
    frame['tokens'] = []
    for idx, (token, pos) in enumerate(result.raw):
        label = 'O'
        if token in time_trigger:
            label = 'TIME'
            frame['relative_time'].append({'token': token, 'index': idx})
        elif token in places:
            label = 'PLACE'
            frame['place'].append({'token': token, 'index': idx})
        elif token in names:
            label = 'PERSON'
            frame['participant'].append({'token': token, 'index': idx})
        frame['tokens'].append({'token': token, 'index': idx, 'pos': pos, 'label': label})

# Output Frames to json

In [ ]:
import json
json.dump(frames, open('frames.json', 'w'), ensure_ascii=False)
print(len(frames))

In [ ]:
frames[1] # After adding 肇嘉嫂 淑英 天壇 to CJ's entities

In [ ]:
frames[2]